![image.png](attachment:image.png)


## Task:

* Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

* Read the file ( Present approach of reading the file )

* Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational     efficiency

* Perform basic validation on data columns : eg: remove special character , white spaces from the col name

* As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of   
  read and write file, column name in YAML

* Validate number of columns and column name of ingested file with YAML.

* Write the file in pipe separated text file (|) in gz format.

* Create a summary of the file:

    Total number of rows,

    total number of columns

    file size
    
# Data Ingestion sample code walkthrough

##
  > Create a utility file
  
  > Config file creation
  
  > Data ingestion pipeline


In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


### Write YAML file

In [12]:
%%writefile file.yaml
file_type: csv
dataset_name: discogs
file_name: discogs
table_name: discogs
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - id
    - release_id
    - status
    - title
    - artist_id
    - artist_name
    - label_name
    - label_id
    - format
    - genre
    - style
    - country
    - release_date
    - notes
    - master_id
    - video_url
    - company_name



Overwriting file.yaml


In [13]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [14]:
config_data['inbound_delimiter']

','

In [15]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'discogs',
 'file_name': 'discogs',
 'table_name': 'discogs',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['id',
  'release_id',
  'status',
  'title',
  'artist_id',
  'artist_name',
  'label_name',
  'label_id',
  'format',
  'genre',
  'style',
  'country',
  'release_date',
  'notes',
  'master_id',
  'video_url',
  'company_name']}

In [11]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("discogs.csv", delimiter=',')
df_sample.head()

FileNotFoundError: ignored

In [ ]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,city,age,Country
0,Delhi,34,India
1,Lima,30,Peru
2,Istanbul,16,Turkey
3,Riyadh,33,Saudi Arabia


In [ ]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['country', 'age']
Following YAML columns are not in the file uploaded ['distance', 'price']


0

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['city', 'age', 'country'], dtype='object')
columns of YAML are: ['city', 'price', 'distance']


In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['country', 'age']
Following YAML columns are not in the file uploaded ['distance', 'price']
validation failed


In [ ]:
pd.read_csv("./test_data.csv")

,city,age,Country
0,Delhi,34,India
1,Lima,30,Peru
2,Istanbul,16,Turkey
3,Riyadh,33,Saudi Arabia


In [ ]:
df

,city,age,country
0,Delhi,34,NaN
1,Lima,30,NaN
2,Istanbul,16,NaN
3,Riyadh,33,NaN
